In [29]:
import os
import pprint
import numpy as np
import cv2
import pydicom
from pathlib import Path

In [42]:
def cropBorders(img, l=0.01, r=0.01, u=0.04, d=0.04):
    
    nrows, ncols = img.shape
    
    l_crop = int(ncols * l)
    r_crop = int(ncols * (1 - r))
    u_crop = int(nrows * u)
    d_crop = int(nrows * (1 - d))
    
    cropped_img = img[u_crop:d_crop, l_crop:r_crop]
    
    return cropped_img

In [3]:
def minMaxNormalise(img):
    
    norm_img = (img - img.min()) / (img.max() - img.min())
    
    return norm_img

In [4]:
def globalBinarise(img, thresh, maxval) :
    
    binarised_img = np.zeros(img.shape, np.uint8)
    binarised_img[img >= thresh] = maxval
    
    return binarised_img

In [5]:
def editMask(mask, ksize=(23, 23), operation="open"):
    
    kernel = cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize = ksize)
    
    if operation == "open":
        edited_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    elif operation == "close":
        edited_mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    edited_mask = cv2.morphologyEx(edited_mask, cv2.MORPH_DILATE, kernel)

    return edited_mask

In [6]:
def sortContoursByArea(contours, reverse=True):

    sorted_contours = sorted(contours, key=cv2.contourArea, reverse=reverse)

    bounding_boxes = [cv2.boundingRect(c) for c in sorted_contours]

    return sorted_contours, bounding_boxes

In [8]:
def xLargestBlobs(mask, top_x=None, reverse=True):
    
    contours, hierarchy = cv2.findContours(
        image=mask, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE
    )
    
    n_contours = len(contours)
    
    if n_contours > 0:
        
        if n_contours < top_x or top_x == None:
            top_x = n_contours

        sorted_contours, bounding_boxes = sortContoursByArea(
            contours=contours, reverse=reverse
        )

        X_largest_contours = sorted_contours[0:top_x]

        to_draw_on = np.zeros(mask.shape, np.uint8)

        X_largest_blobs = cv2.drawContours(
            image=to_draw_on,  
            contours=X_largest_contours,  
            contourIdx=-1,
            color=1,  
            thickness=-1, 
        )

    return n_contours, X_largest_blobs
        

In [9]:
def applyMask(img, mask):

    masked_img = img.copy()
    masked_img[mask == 0] = 0

    return masked_img

In [10]:
 def checkLRFlip(mask):

    nrows, ncols = mask.shape
    x_center = ncols // 2
    y_center = nrows // 2

    col_sum = mask.sum(axis=0)
    row_sum = mask.sum(axis=1)

    left_sum = sum(col_sum[0:x_center])
    right_sum = sum(col_sum[x_center:-1])

    if left_sum < right_sum:
        LR_flip = True
    else:
        LR_flip = False

    return LR_flip

In [11]:
def makeLRFlip(img):

    flipped_img = np.fliplr(img)

    return flipped_img

In [12]:
def clahe(img, clip=2.0, tile=(8, 8)):

    img = cv2.normalize(
        img,
        None,
        alpha=0,
        beta=255,
        norm_type=cv2.NORM_MINMAX,
        dtype=cv2.CV_32F,
    )
    img_uint8 = img.astype("uint8")

    clahe_create = cv2.createCLAHE(clipLimit=clip, tileGridSize=tile)
    clahe_img = clahe_create.apply(img_uint8)

    return clahe_img

In [13]:
def pad(img):

    nrows, ncols = img.shape

    if nrows != ncols:

        if ncols < nrows:
            
            target_shape = (nrows, nrows)
        
        elif nrows < ncols:
            
            target_shape = (ncols, ncols)

        padded_img = np.zeros(shape=target_shape)
        padded_img[:nrows, :ncols] = img

    elif nrows == ncols:

        padded_img = img

    return padded_img

In [14]:

def fullMammoPreprocess(
    img,
    l,
    r,
    d,
    u,
    thresh,
    maxval,
    ksize,
    operation,
    reverse,
    top_x,
    clip,
    tile,
):

    # Step 1: Initial crop.
    cropped_img = cropBorders(img=img, l=l, r=r, d=d, u=u)

    # Step 2: Min-max normalise.
    norm_img = minMaxNormalise(img=cropped_img)

    # Step 3: Remove artefacts.
    binarised_img = globalBinarise(img=norm_img, thresh=thresh, maxval=maxval)
    edited_mask = editMask(
        mask=binarised_img, ksize=(ksize, ksize), operation=operation
    )
    _, xlargest_mask = xLargestBlobs(mask=edited_mask, top_x=top_x, reverse=reverse)

    masked_img = applyMask(img=norm_img, mask=xlargest_mask)

    # Step 4: Horizontal flip.
    lr_flip = checkLRFlip(mask=xlargest_mask)
    if lr_flip:
        flipped_img = makeLRFlip(img=masked_img)
    elif not lr_flip:
        flipped_img = masked_img

    # Step 5: CLAHE enhancement.
    clahe_img = clahe(img=flipped_img, clip=clip, tile=(tile, tile))

    # Step 6: pad.
    padded_img = pad(img=clahe_img)
    padded_img = cv2.normalize(
        padded_img,
        None,
        alpha=0,
        beta=255,
        norm_type=cv2.NORM_MINMAX,
        dtype=cv2.CV_32F,
    )

    # Step 7: Min-max normalise.
    img_pre = minMaxNormalise(img=padded_img)

    return img_pre, lr_flip

In [44]:
def maskPreprocess(mask, lr_flip):

    # Step 1: Initial crop.
    mask = cropBorders(img=mask)

    # Step 2: Horizontal flip.
    if lr_flip:
        mask = makeLRFlip(img=mask)

    # Step 3: Pad.
    mask_pre = pad(img=mask)

    return mask_pre

In [45]:
def sumMasks(mask_list):

    summed_mask = np.zeros(mask_list[0].shape)

    for arr in mask_list:
        summed_mask = np.add(summed_mask, arr)

    _, summed_mask_bw = cv2.threshold(
        src=summed_mask, thresh=1, maxval=255, type=cv2.THRESH_BINARY
    )

    return summed_mask_bw

In [32]:
config_massTrainPre = {
    "paths": {
        "input": "data\Mass\Train\ALL",
        "output_full": "data\Mass\Train\FULL",
        "output_mask": "data\Mass\Train\MASK"
    },
    "output_format": ".png",
    "cropBorders": {
        "l": 0.01,
        "r": 0.01,
        "u": 0.04,
        "d": 0.04
    },
    "globalBinarise": {
        "thresh": 0.1,
        "maxval": 1.0
    },
    "editMask": {
        "ksize": 23,
        "operation": "open"
    },
    "sortContourByArea": {
        "reverse": True
    },
    "xLargestBlobs": {
        "top_x": 1
    },
    "clahe": {
        "clip": 2.0,
        "tile": 8
    }
}

In [33]:
input_path = config_massTrainPre["paths"]["input"]
output_full_path = config_massTrainPre["paths"]["output_full"]
output_mask_path = config_massTrainPre["paths"]["output_mask"]

output_format = config_massTrainPre["output_format"]

In [34]:
dcm_paths = []
for curdir, dirs, files in os.walk(input_path):
    files.sort()
    for f in files:
        if f.endswith(".dcm"):
            dcm_paths.append(os.path.join(curdir, f))

In [36]:
fullmamm_paths = [f for f in dcm_paths if ("FULL" in f)]
mask_paths = [f for f in dcm_paths if ("MASK" in f)]

In [46]:
count = 0
for fullmamm_path in fullmamm_paths:
    ds = pydicom.dcmread(fullmamm_path)
    
    patient_id = ds.PatientID
    
    patient_id = patient_id.replace(".dcm", "")
    patient_id = patient_id.replace("Mass-Training_", "")

    fullmamm = ds.pixel_array

    l = config_massTrainPre["cropBorders"]["l"]
    r = config_massTrainPre["cropBorders"]["r"]
    u = config_massTrainPre["cropBorders"]["u"]
    d = config_massTrainPre["cropBorders"]["d"]
    thresh = config_massTrainPre["globalBinarise"]["thresh"]
    maxval = config_massTrainPre["globalBinarise"]["maxval"]
    ksize = config_massTrainPre["editMask"]["ksize"]
    operation = config_massTrainPre["editMask"]["operation"]
    reverse = config_massTrainPre["sortContourByArea"]["reverse"]
    top_x = config_massTrainPre["xLargestBlobs"]["top_x"]
    clip = config_massTrainPre["clahe"]["clip"]
    tile = config_massTrainPre["clahe"]["tile"]

    fullmamm_pre, lr_flip = fullMammoPreprocess(
            img=fullmamm,
            l=l,
            r=r,
            u=u,
            d=d,
            thresh=thresh,
            maxval=maxval,
            ksize=ksize,
            operation=operation,
            reverse=reverse,
            top_x=top_x,
            clip=clip,
            tile=tile,
        )

    fullmamm_pre_norm = cv2.normalize(
            fullmamm_pre,
            None,
            alpha=0,
            beta=255,
            norm_type=cv2.NORM_MINMAX,
            dtype=cv2.CV_32F,
        )

    save_filename = (
            os.path.basename(fullmamm_path).replace(".dcm", "")
            + "_PRE"
            + output_format
        )
    save_path = os.path.join(output_full_path, save_filename)
    cv2.imwrite(save_path, fullmamm_pre_norm)
    print(f"DONE FULL: {fullmamm_path}")

    mask_path = [mp for mp in mask_paths if patient_id in mp]

    for mp in mask_path:
        
        mask_ds = pydicom.dcmread(mp)
        mask = mask_ds.pixel_array
        
        mask_pre = maskPreprocess(mask=mask, lr_flip=lr_flip)

        save_filename = (
                os.path.basename(mp).replace(".dcm", "") + "___PRE" + output_format
            )
        save_path = os.path.join(output_mask_path, save_filename)
        cv2.imwrite(save_path, mask_pre)

        print(f"DONE MASK: {mp}")

    count += 1

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00001_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00001_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00001_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00001_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00004_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00004_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00004_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00004_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00004_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00004_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00009_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00009_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00009_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00009_RIGHT_MLO_MAS

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00080_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00080_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00081_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00081_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00081_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00081_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00086_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00086_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00086_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00086_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00090_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00090_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00090_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00090_LEFT_MLO

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00166_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00166_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00166_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00166_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00169_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00169_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00172_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00172_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00172_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00172_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00174_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00174_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00174_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00174_RIGHT_MLO

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00259_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00259_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00259_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00259_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00264_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00264_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00264_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00264_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00265_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00265_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00265_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00265_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00273_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00273_LEFT_CC_MAS

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00354_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00354_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00354_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00354_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00356_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00356_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00361_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00361_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00363_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00363_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00363_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00363_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_00366_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_00366_RIGHT_CC_MASK_1.

...

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01544_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01544_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01544_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01544_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01553_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01553_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01556_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01556_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01557_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01557_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01557_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01557_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01559_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01559_RIGHT_M

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01634_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01634_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01635_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01635_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01635_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01635_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01635_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01635_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01635_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01635_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01636_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01636_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01637_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01637_LEFT_MLO_M

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01702_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01702_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01702_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01702_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01710_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01710_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01710_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01710_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01712_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01712_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01712_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01712_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01712_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01712_RIGHT_CC_MASK_

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01798_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01798_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01798_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01798_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01799_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01799_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01799_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01799_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01801_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01801_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01801_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01801_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01802_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01802_LEFT_MLO_MASK_

DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01888_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01888_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01889_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01889_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01889_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01889_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01890_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01890_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01908_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01908_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01946_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01946_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Train\ALL\Mass-Training_P_01946_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Train\ALL\Mass-Training_P_01946_RIGHT_MLO

In [47]:
count

1231

In [49]:
config_massTestPre = {
    "paths": {
        "input": "data\Mass\Test\ALL",
        "output_full": "data\Mass\Test\FULL",
        "output_mask": "data\Mass\Test\MASK"
    },
    "output_format": ".png",
    "cropBorders": {
        "l": 0.01,
        "r": 0.01,
        "u": 0.04,
        "d": 0.04
    },
    "globalBinarise": {
        "thresh": 0.1,
        "maxval": 1.0
    },
    "editMask": {
        "ksize": 23,
        "operation": "open"
    },
    "sortContourByArea": {
        "reverse": True
    },
    "xLargestBlobs": {
        "top_x": 1
    },
    "clahe": {
        "clip": 2.0,
        "tile": 8
    }
}

In [50]:
input_path = config_massTestPre["paths"]["input"]
output_full_path = config_massTestPre["paths"]["output_full"]
output_mask_path = config_massTestPre["paths"]["output_mask"]

output_format = config_massTestPre["output_format"]

dcm_paths = []
for curdir, dirs, files in os.walk(input_path):
    files.sort()
    for f in files:
        if f.endswith(".dcm"):
            dcm_paths.append(os.path.join(curdir, f))
            
fullmamm_paths = [f for f in dcm_paths if ("FULL" in f)]
mask_paths = [f for f in dcm_paths if ("MASK" in f)]

count = 0
for fullmamm_path in fullmamm_paths:
    ds = pydicom.dcmread(fullmamm_path)
    
    patient_id = ds.PatientID
    
    patient_id = patient_id.replace(".dcm", "")
    patient_id = patient_id.replace("Mass-Test_", "")

    fullmamm = ds.pixel_array

    l = config_massTestPre["cropBorders"]["l"]
    r = config_massTestPre["cropBorders"]["r"]
    u = config_massTestPre["cropBorders"]["u"]
    d = config_massTestPre["cropBorders"]["d"]
    thresh = config_massTestPre["globalBinarise"]["thresh"]
    maxval = config_massTestPre["globalBinarise"]["maxval"]
    ksize = config_massTestPre["editMask"]["ksize"]
    operation = config_massTestPre["editMask"]["operation"]
    reverse = config_massTestPre["sortContourByArea"]["reverse"]
    top_x = config_massTestPre["xLargestBlobs"]["top_x"]
    clip = config_massTestPre["clahe"]["clip"]
    tile = config_massTestPre["clahe"]["tile"]

    fullmamm_pre, lr_flip = fullMammoPreprocess(
            img=fullmamm,
            l=l,
            r=r,
            u=u,
            d=d,
            thresh=thresh,
            maxval=maxval,
            ksize=ksize,
            operation=operation,
            reverse=reverse,
            top_x=top_x,
            clip=clip,
            tile=tile,
        )

    fullmamm_pre_norm = cv2.normalize(
            fullmamm_pre,
            None,
            alpha=0,
            beta=255,
            norm_type=cv2.NORM_MINMAX,
            dtype=cv2.CV_32F,
        )

    save_filename = (
            os.path.basename(fullmamm_path).replace(".dcm", "")
            + "_PRE"
            + output_format
        )
    save_path = os.path.join(output_full_path, save_filename)
    cv2.imwrite(save_path, fullmamm_pre_norm)
    print(f"DONE FULL: {fullmamm_path}")

    mask_path = [mp for mp in mask_paths if patient_id in mp]

    for mp in mask_path:
        
        mask_ds = pydicom.dcmread(mp)
        mask = mask_ds.pixel_array
        
        mask_pre = maskPreprocess(mask=mask, lr_flip=lr_flip)

        save_filename = (
                os.path.basename(mp).replace(".dcm", "") + "___PRE" + output_format
            )
        save_path = os.path.join(output_mask_path, save_filename)
        cv2.imwrite(save_path, mask_pre)

        print(f"DONE MASK: {mp}")

    count += 1

DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00016_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00016_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00016_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00016_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00017_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00017_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00017_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00017_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00032_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00032_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00032_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00032_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00037_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00037_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00037_RIGHT_MLO_FULL.d

DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00296_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00296_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00296_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00296_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00324_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00324_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00324_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00324_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00340_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00340_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00343_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00343_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00343_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00343_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00343_RIGHT_CC_FULL.dcm


DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00524_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00524_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00533_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00533_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00533_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00533_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00544_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00544_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00544_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00544_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00576_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00576_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00576_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00576_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00587_RIGHT_CC_FULL.dcm
DO

DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00893_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00893_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00922_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00922_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00922_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00922_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00924_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00924_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00928_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00928_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00928_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00928_RIGHT_MLO_MASK_1.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00928_RIGHT_MLO_MASK_2.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_00932_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_00932_LEFT_MLO_

DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01251_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01251_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01251_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01254_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01254_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01254_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01254_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01257_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01257_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01257_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01257_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01277_RIGHT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01277_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01277_RIGHT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01277_RIGHT_MLO_

DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01578_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01578_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01590_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01590_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01590_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01590_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01595_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01595_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01595_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01595_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01599_LEFT_CC_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01599_LEFT_CC_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01599_LEFT_MLO_FULL.dcm
DONE MASK: data\Mass\Test\ALL\Mass-Test_P_01599_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Mass\Test\ALL\Mass-Test_P_01605_LEFT_CC_FULL.dcm
DON

In [52]:
config_calcTrainPre = {
    "paths": {
        "input": "data\Calc\Train\ALL",
        "output_full": "data\Calc\Train\FULL",
        "output_mask": "data\Calc\Train\MASK"
    },
    "output_format": ".png",
    "cropBorders": {
        "l": 0.01,
        "r": 0.01,
        "u": 0.04,
        "d": 0.04
    },
    "globalBinarise": {
        "thresh": 0.1,
        "maxval": 1.0
    },
    "editMask": {
        "ksize": 23,
        "operation": "open"
    },
    "sortContourByArea": {
        "reverse": True
    },
    "xLargestBlobs": {
        "top_x": 1
    },
    "clahe": {
        "clip": 2.0,
        "tile": 8
    }
}

In [53]:
input_path = config_calcTrainPre["paths"]["input"]
output_full_path = config_calcTrainPre["paths"]["output_full"]
output_mask_path = config_calcTrainPre["paths"]["output_mask"]

output_format = config_calcTrainPre["output_format"]

dcm_paths = []
for curdir, dirs, files in os.walk(input_path):
    files.sort()
    for f in files:
        if f.endswith(".dcm"):
            dcm_paths.append(os.path.join(curdir, f))
            
fullmamm_paths = [f for f in dcm_paths if ("FULL" in f)]
mask_paths = [f for f in dcm_paths if ("MASK" in f)]

count = 0
for fullmamm_path in fullmamm_paths:
    ds = pydicom.dcmread(fullmamm_path)
    
    patient_id = ds.PatientID
    
    patient_id = patient_id.replace(".dcm", "")
    patient_id = patient_id.replace("Calc-Training_", "")

    fullmamm = ds.pixel_array

    l = config_calcTrainPre["cropBorders"]["l"]
    r = config_calcTrainPre["cropBorders"]["r"]
    u = config_calcTrainPre["cropBorders"]["u"]
    d = config_calcTrainPre["cropBorders"]["d"]
    thresh = config_calcTrainPre["globalBinarise"]["thresh"]
    maxval = config_calcTrainPre["globalBinarise"]["maxval"]
    ksize = config_calcTrainPre["editMask"]["ksize"]
    operation = config_calcTrainPre["editMask"]["operation"]
    reverse = config_calcTrainPre["sortContourByArea"]["reverse"]
    top_x = config_calcTrainPre["xLargestBlobs"]["top_x"]
    clip = config_calcTrainPre["clahe"]["clip"]
    tile = config_calcTrainPre["clahe"]["tile"]

    fullmamm_pre, lr_flip = fullMammoPreprocess(
            img=fullmamm,
            l=l,
            r=r,
            u=u,
            d=d,
            thresh=thresh,
            maxval=maxval,
            ksize=ksize,
            operation=operation,
            reverse=reverse,
            top_x=top_x,
            clip=clip,
            tile=tile,
        )

    fullmamm_pre_norm = cv2.normalize(
            fullmamm_pre,
            None,
            alpha=0,
            beta=255,
            norm_type=cv2.NORM_MINMAX,
            dtype=cv2.CV_32F,
        )

    save_filename = (
            os.path.basename(fullmamm_path).replace(".dcm", "")
            + "_PRE"
            + output_format
        )
    save_path = os.path.join(output_full_path, save_filename)
    cv2.imwrite(save_path, fullmamm_pre_norm)
    print(f"DONE FULL: {fullmamm_path}")

    mask_path = [mp for mp in mask_paths if patient_id in mp]

    for mp in mask_path:
        
        mask_ds = pydicom.dcmread(mp)
        mask = mask_ds.pixel_array
        
        mask_pre = maskPreprocess(mask=mask, lr_flip=lr_flip)

        save_filename = (
                os.path.basename(mp).replace(".dcm", "") + "___PRE" + output_format
            )
        save_path = os.path.join(output_mask_path, save_filename)
        cv2.imwrite(save_path, mask_pre)

        print(f"DONE MASK: {mp}")

    count += 1

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00005_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00005_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00005_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00005_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00007_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00007_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00007_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00007_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00008_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00008_LEFT_CC_MASK_1.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00008_LEFT_CC_MASK_2.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00008_LEFT_CC_MASK_3.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00008_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00008_LEFT_MLO_MASK_

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00062_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00062_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00063_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00063_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00063_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00063_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00071_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00071_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00071_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00071_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00078_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00078_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00078_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00078_LEFT_MLO_M

DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00162_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00162_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00162_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00179_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00179_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00179_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00179_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00181_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00181_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00181_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00181_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00182_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00182_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00182_LEFT_MLO_FULL

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00282_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00282_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00282_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00282_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00284_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00284_LEFT_CC_MASK_1.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00284_LEFT_CC_MASK_2.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00284_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00284_LEFT_MLO_MASK_1.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00284_LEFT_MLO_MASK_2.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00288_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00288_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00288_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00288_RIGHT_MLO_MASK

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00378_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00378_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00378_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00378_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00380_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00380_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00380_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00380_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00385_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00385_LEFT_CC_MASK_1.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00385_LEFT_CC_MASK_2.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00385_LEFT_CC_MASK_3.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_00385_LEFT_CC_MASK_4.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_00385_LEFT_MLO_FULL

...

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01655_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01655_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01655_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01655_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01659_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01659_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01659_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01659_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01660_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01660_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01660_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01660_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01664_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01664_LEFT_CC_MAS

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01748_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01748_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01750_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01750_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01750_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01750_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01756_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01756_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01756_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01756_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01759_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01759_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01759_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01759_RIGHT_MLO_MAS

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_CC_MASK_1.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_CC_MASK_2.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_CC_MASK_3.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_CC_MASK_4.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_CC_MASK_5.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO_MASK_1.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO_MASK_2.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO_MASK_3.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO_MASK_4.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO_MASK_5.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO_MASK_6.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_01838_LEFT_MLO

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02226_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02226_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02232_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02232_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02232_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02232_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02234_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02234_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02234_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02234_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02259_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02259_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02259_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02259_LEFT_MLO_MASK_1.dc

DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02508_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02508_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02518_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02518_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02518_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02518_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02522_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02522_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02522_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02522_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02559_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02559_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Train\ALL\Calc-Training_P_02559_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Train\ALL\Calc-Training_P_02559_RIGHT_M

In [54]:
config_calcTestPre = {
    "paths": {
        "input": "data\Calc\Test\ALL",
        "output_full": "data\Calc\Test\FULL",
        "output_mask": "data\Calc\Test\MASK"
    },
    "output_format": ".png",
    "cropBorders": {
        "l": 0.01,
        "r": 0.01,
        "u": 0.04,
        "d": 0.04
    },
    "globalBinarise": {
        "thresh": 0.1,
        "maxval": 1.0
    },
    "editMask": {
        "ksize": 23,
        "operation": "open"
    },
    "sortContourByArea": {
        "reverse": True
    },
    "xLargestBlobs": {
        "top_x": 1
    },
    "clahe": {
        "clip": 2.0,
        "tile": 8
    }
}

In [55]:
input_path = config_calcTestPre["paths"]["input"]
output_full_path = config_calcTestPre["paths"]["output_full"]
output_mask_path = config_calcTestPre["paths"]["output_mask"]

output_format = config_calcTestPre["output_format"]

dcm_paths = []
for curdir, dirs, files in os.walk(input_path):
    files.sort()
    for f in files:
        if f.endswith(".dcm"):
            dcm_paths.append(os.path.join(curdir, f))
            
fullmamm_paths = [f for f in dcm_paths if ("FULL" in f)]
mask_paths = [f for f in dcm_paths if ("MASK" in f)]

count = 0
for fullmamm_path in fullmamm_paths:
    ds = pydicom.dcmread(fullmamm_path)
    
    patient_id = ds.PatientID
    
    patient_id = patient_id.replace(".dcm", "")
    patient_id = patient_id.replace("Calc-Test_", "")

    fullmamm = ds.pixel_array

    l = config_calcTestPre["cropBorders"]["l"]
    r = config_calcTestPre["cropBorders"]["r"]
    u = config_calcTestPre["cropBorders"]["u"]
    d = config_calcTestPre["cropBorders"]["d"]
    thresh = config_calcTestPre["globalBinarise"]["thresh"]
    maxval = config_calcTestPre["globalBinarise"]["maxval"]
    ksize = config_calcTestPre["editMask"]["ksize"]
    operation = config_calcTestPre["editMask"]["operation"]
    reverse = config_calcTestPre["sortContourByArea"]["reverse"]
    top_x = config_calcTestPre["xLargestBlobs"]["top_x"]
    clip = config_calcTestPre["clahe"]["clip"]
    tile = config_calcTestPre["clahe"]["tile"]

    fullmamm_pre, lr_flip = fullMammoPreprocess(
            img=fullmamm,
            l=l,
            r=r,
            u=u,
            d=d,
            thresh=thresh,
            maxval=maxval,
            ksize=ksize,
            operation=operation,
            reverse=reverse,
            top_x=top_x,
            clip=clip,
            tile=tile,
        )

    fullmamm_pre_norm = cv2.normalize(
            fullmamm_pre,
            None,
            alpha=0,
            beta=255,
            norm_type=cv2.NORM_MINMAX,
            dtype=cv2.CV_32F,
        )

    save_filename = (
            os.path.basename(fullmamm_path).replace(".dcm", "")
            + "_PRE"
            + output_format
        )
    save_path = os.path.join(output_full_path, save_filename)
    cv2.imwrite(save_path, fullmamm_pre_norm)
    print(f"DONE FULL: {fullmamm_path}")

    mask_path = [mp for mp in mask_paths if patient_id in mp]

    for mp in mask_path:
        
        mask_ds = pydicom.dcmread(mp)
        mask = mask_ds.pixel_array
        
        mask_pre = maskPreprocess(mask=mask, lr_flip=lr_flip)

        save_filename = (
                os.path.basename(mp).replace(".dcm", "") + "___PRE" + output_format
            )
        save_path = os.path.join(output_mask_path, save_filename)
        cv2.imwrite(save_path, mask_pre)

        print(f"DONE MASK: {mp}")

    count += 1

DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00038_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00038_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00038_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00038_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00038_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00038_RIGHT_CC_MASK_1.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00038_RIGHT_CC_MASK_2.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00038_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00038_RIGHT_MLO_MASK_1.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00038_RIGHT_MLO_MASK_2.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00041_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00041_LEFT_CC_MASK_2.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00041_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00041_LEFT_MLO_MASK_2.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00077_LEFT_CC_FULL.

DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00390_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00390_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00390_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00390_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00397_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00397_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00402_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00402_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00402_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00402_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00403_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00403_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00403_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00403_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00460_LEFT_CC_FU

DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00857_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00857_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00857_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00857_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00876_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00876_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00876_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00876_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00879_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00879_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00879_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00879_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00905_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_00905_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_00905_RIGHT_MLO_FU

DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01318_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01318_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01320_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01320_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01352_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01352_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01352_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01352_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01353_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01353_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01353_LEFT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01353_LEFT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01390_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01390_RIGHT_CC_MASK_1.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01390_RIGHT_CC_MASK_2.dc

DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01670_LEFT_MLO_MASK_2.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01670_LEFT_MLO_MASK_3.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01711_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01711_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01711_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01711_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01713_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01713_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01728_LEFT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01728_LEFT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01743_RIGHT_CC_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01743_RIGHT_CC_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01743_RIGHT_MLO_FULL.dcm
DONE MASK: data\Calc\Test\ALL\Calc-Test_P_01743_RIGHT_MLO_MASK_1.dcm
DONE FULL: data\Calc\Test\ALL\Calc-Test_P_01752_LEFT_CC_